In [29]:
!python -m spacy download en_core_web_sm
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------ --------------------------------- 2.1/12.8 MB 11.8 MB/s eta 0:00:01
     -------------- ------------------------- 4.7/12.8 MB 11.9 MB/s eta 0:00:01
     ---------------------- ----------------- 7.3/12.8 MB 11.9 MB/s eta 0:00:01
     ------------------------------ -------- 10.0/12.8 MB 11.9 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 12.0 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 11.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     --------------------------------------- 1.8/400.7 MB 11.2 MB/s eta 0:00:36
     --------------------------------------- 4.5/400.7 MB 11.7 MB/s eta 0:00:34
      -------------------------------------- 6.8/400.7 MB 11.7 MB/s eta 0:00:34
      ----------------------------

In [30]:
text = "What video sharing service did Steve Chen, Chad Hurley, and Jawed Karim create in 2005 at America?"

import spacy

nlp = spacy.load("en_core_web_lg")
doc = nlp(text)
 
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [31]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

In [32]:
doc = nlp("India beat Bangladesh")
displacy.render(doc, style="ent", jupyter=True)
doc.ents


(India, Bangladesh)

In [33]:
import json

with open('dataset/sample2.json', 'r') as f:
    data = json.load(f)

In [40]:
training_data = []
for example in data['examples']:
    temp_dict = {}
    temp_dict['text'] = example['content']
    temp_dict['entities'] = []
    for annotation in example['annotations']:
        start = annotation['start']
        end = annotation['end']
        label = annotation['tag_name'].upper()
        temp_dict['entities'].append((start, end, label))
    training_data.append(temp_dict)


print(training_data)

[{'text': 'Govardhana K Senior Software Engineer  Bengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/ b2de315d95905b68  Total IT experience 5 Years 6 Months Cloud Lending Solutions INC 4 Month • Salesforce Developer Oracle 5 Years 2 Month • Core Java Developer Languages Core Java, Go Lang Oracle PL-SQL programming, Sales Force Developer with APEX.  Designations & Promotions  Willing to relocate: Anywhere  WORK EXPERIENCE  Senior Software Engineer  Cloud Lending Solutions -  Bangalore, Karnataka -  January 2018 to Present  Present  Senior Consultant  Oracle -  Bangalore, Karnataka -  November 2016 to December 2017  Staff Consultant  Oracle -  Bangalore, Karnataka -  January 2014 to October 2016  Associate Consultant  Oracle -  Bangalore, Karnataka -  November 2012 to December 2013  EDUCATION  B.E in Computer Science Engineering  Adithya Institute of Technology -  Tamil Nadu  September 2008 to June 2012  https://www.indeed.com/r/Govardhana-K/b2de315d95905b68?

In [41]:
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
doc_bin = DocBin()

from spacy.util import filter_spans

for training_example in tqdm(training_data):
     text = training_example['text']
     labels = training_example['entities']
     doc = nlp.make_doc(text)
     ents = []
     for start, end, label in labels:
         span = doc.char_span(start, end, label=label, alignment_mode="contract")
         if span is None:
             print("Skipping entity")
         else:
             ents.append(span)
     filtered_ents = filter_spans(ents)
     doc.ents = filtered_ents
     doc_bin.add(doc)

doc_bin.to_disk("train.spacy")


100%|██████████| 1/1 [00:00<00:00, 195.74it/s]


In [42]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [43]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy 



ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    221.71    0.00    0.00    0.00    0.00
200     200       8098.72   6701.57  100.00  100.00  100.00    1.00
400     400          0.00      0.00  100.00  100.00  100.00    1.00
600     600          0.00      0.00  100.00  100.00  100.00    1.00
800     800          0.00      0.00  100.00  100.00  100.00    1.00
1000    1000          0.00      0.00  100.00  100.00  100.00    1.00
1200    1200          0.00      0.00  100.00  100.00  100.00    1.00
1400    1400          0.00      0.00  100.00  100.00  100.00    1.00
1600    1600          0.00      0

In [48]:
nlp_ner = spacy.load("model-best")

# Render the named entities with custom colors
text1='Vineeth Vijayan "Store Executive" - Orange City Hospital & Research Institute  Nagpur, Maharashtra, Maharashtra - Email me on Indeed: indeed.com/r/Vineeth-Vijayan/ ee84e7ea0695181f  I have over 2 years of experience working as a "Store Executive" in a reputed big organisation based in India. I am a hard worker and a very good learner. I aspire to grow in my career by joining a good organisation.  I am good at work and people management. I possess good computer knowledge. I have underwent training in courses like MS Excel Level 1, 2 &amp; 3. MS Word &amp; Powerpoint. I have experience working in ERP. I am a certified ISO 27001: 2013 Internal Auditor.  Willing to relocate: Anywhere  WORK EXPERIENCE  "Store Executive"  Orange City Hospital & Research Institute -  Nagpur, Maharashtra -  March 2016 to Present  Job Profile: - 1. Keep records to maintain inventory control, cost containment and to assure proper stock levels. 2. Rotate stock and arrange for disposal of surpluses. 3. Plan and perform work that involves ordering, receiving, inspecting, returning, unloading, shelving, packing, labeling and maintaining a perpetual inventory of items. 4. Coordinate freight handling, equipment moving and minor repairs. 5. Perform peripheral site-specific duties as required and instructed by supervisors. 6. Monitoring and updating stock details etc. in Mednet store software. Use ABC, VED &amp; FSN Stock Analysis methods when required. 7. Report any stock discrepancy to supervisor for appropriate action.  "HR Officer "  Gulf Warehousing Company Qatar - Agility Group -  January 2012 to July 2015  a multinational company catering Customers in the field of Warehousing, Freight Forwarding, Records Management, Courier and International Relocation Services.  Job Profile: - 1. Deliver new hires orientation program to enable new hires to adjust themselves to new working environment and facilitating newcomers joining formalities. 2. Design and develop Training Calendar and budget for all type of training programs, for the year ahead for all designations within the company, according to the needs of the business and Training Need Analysis survey results.  https://www.indeed.com/r/Vineeth-Vijayan/ee84e7ea0695181f?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Vineeth-Vijayan/ee84e7ea0695181f?isid=rex-download&ikw=download-top&co=IN   3. Sourcing and coordinating with external vendors for training that are requested by line managers/departments. 4. Maintaining employee training records and updating it on a regular basis. 5. Assist Supervisors in developing and implementing changes in HR policies and procedures. 6. Coordinate in conducting employee engagement programs. 7. Support departments in complying with ISO Quality Management requirements and facilitate various HR related welfare activities.  "Training Coordinator"  M-Squared Software & Services - Kerala -  Kerala, IN -  July 2010 to December 2011  a US based company with headquarters in Nevada, Las Vegas that deals with security gadgets, medical transcription and software development.  Job Profile: - 1. Perform regular office administration duties that includes: Office infrastructure management, housekeeping supervision, responding and sending e-mails to head office and clients, monitoring time-in &amp; time-out of employees, managing employee leave status and office cash management. 2. Handle walk-in enquires, attend telephone calls and conduct course induction sessions for new joiners. 3. Train the students in work related processes. 4. Anchoring and conducting course related seminars in well-established educational institutions. 5. Manage training materials that are stipulated within the company regulations. 6. Formulating training programs and sending it to training manager and to all concerned departments for approval. 7. Preparing tools for training. 8. Organize the training that includes: distribution of training materials, monitor trainees in training and support trainers in training. 9. Record training plans and training programs. 10. Maintain employees and students database. 11. Record results of evaluation and training. 12. Archive records of training, including course content, the training, number of students, results, and feedback. 13. Conduct team huddles/counseling sessions for trainees when required.  "Manager"  Nair IT Pvt. Ltd -  Nagpur, Maharashtra -  September 2008 to July 2010  Client: Accentia Oak Technologies.  Job Profile: - 1. To manage the day-to-day planning, operation and problem-solving of the team to meet the required service level components, quality and productivity targets of the client.    2. Delivery of team statistics, quality and productivity targets &amp; indicators, taking note any downtime during the day and report it to the management level superiors. 3. Operational Management: Managing the floor, adherence to schedule and monitoring of attendance. Submits daily report to the management level superiors. 4. Monitoring the portal periodically, prioritizing jobs according to TAT, and assigning jobs to correct work pool. 5. Coaching and relaying feedback to MTs/Editors. 6. To motivate the team to achieve the client targets. Update the team on the status of work at the start and end of each shift. 7. Compiling reports on team\'s performance and client feedback.  "Process Executive" in Infosys BPO Limited, Bangalore  CISCO -  Bengaluru, Karnataka -  January 2006 to July 2008  Job Profile: - 1. To handle end to end order placement process of Cisco. 2. Book orders for customers as per Cisco\'s booking policies. 3. Customer advocacy. 4. Check and avail special discounts, services, licensing, upgrades etc., to the customers. 5. Receive customer feedback on products, services etc., and resolve them if any. 6. To work along with logistic team to ensure that the products reach precise destinations safely. 7. Provide round the clock 24/7 customer service to the client.  "Sr. Medical Transcriptionist"  DTS Information Systems Pvt. Ltd -  Bengaluru, Karnataka -  December 2004 to January 2006  in DTS Information Systems Pvt. Ltd, Bangalore (A wholly owned subsidiary of DTS America Inc., Tennessee, USA) from Dec 2004 - Jan 2006. Client: Vanderbilt University Medical Center, Nashville, Tennessee.  Job Profile: - 1. To transcribe documents pertaining to patient\'s summary of illness, diseases and diagnostic procedures etc. to avail medical insurance claims as per American transcription guidelines. 2. To ensure that finished end document meets the clients TAT and Quality requirements. 3. To counsel MT\'s regarding their errors etc and if needed conduct team huddles. Computer Exposure  Operating System: Windows XP Microsoft Office Package: MS Excel, MS Word, MS Access and Power Point. Experience in working on Oracle.  ❖ Completed MS Excel Level 1, 2 &amp; 3 Courses from New Horizons Computer Training Center, Qatar.    EDUCATION  B.Sc. in Microbiology in Microbiology  S.N.G College, Bharathiar University  April 2004  SKILLS  Storekeeping / Inventory Management / Purchase (2 years), People & Time Management (2 years)  ADDITIONAL INFORMATION  Competencies  ❖ Effective communications skills, excellent interpersonal skills, good time management skills, results-oriented individual and a very good team player. ❖ Sincere, hardworking and willing to take up challenging assignments.  Other Qualifications  ❖ ISO 27001: 2013 Internal Auditor Certification from Coms Vantage Consultancy, Qatar in October 2014 ❖ ISO 27001: 2013 Awareness Certification from Coms Vantage Consultancy, Qatar in September 2014 ❖ Carriage of Dangerous Goods by Road Certification in 2012 from IRU Academy in tie up with Mowasalat - State of Qatar owned Transport Company. ❖ First class in Hindi Parichaya Examination conducted by Dakshina Bharat Hindi Prachar Sabha. ❖ Second class in Prathamic Examination conducted by Dakshina Bharat Hindi Prachar Sabha.'
doc=nlp_ner(text1)
displacy.render(doc, style="ent", jupyter=True)
for ent in doc.ents:
    print(f'{ent.label_.upper():{30}}- {ent.text}')

GRADUATION YEAR               - "
SKILLS                        - 3. MS Word &amp; Powerpoint. I have experience working in ERP. I am a certified ISO 27001: 2013 Internal Auditor.  Willing to relocate: Anywhere  WORK EXPERIENCE  
COLLEGE NAME                  - "Store Executive"
COMPANIES WORKED AT           - Orange
DESIGNATION                   - Rotate stock and
COLLEGE NAME                  - Coordinate freight handling, equipment moving
SKILLS                        - software. Use ABC, VED &amp; FSN Stock Analysis methods when required. 7. Report any stock discrepancy to supervisor for appropriate action.  
COLLEGE NAME                  - "HR Officer
COMPANIES WORKED AT           - Company Qatar
GRADUATION YEAR               - 2012
COLLEGE NAME                  - a multinational company catering
COMPANIES WORKED AT           - Customers in the
SKILLS                        - all type of training programs, for the year ahead for all designations within the company, according to th